In [136]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import re
from matplotlib.lines import Line2D
from datetime import datetime, date, time, timedelta
%matplotlib inline

In [187]:
df=pd.read_csv('/Users/ekaterinakotova/Desktop/2 семестр/МО/sem3/Titanic.csv')

In [111]:
df.head()

,Name,PClass,Age,Sex,Survived
0,"Allen, Miss Elisabeth Walton",1st,29.00,female,1
1,"Allison, Miss Helen Loraine",1st,2.00,female,0
2,"Allison, Mr Hudson Joshua Creighton",1st,30.00,male,0
3,"Allison, Mrs Hudson JC (Bessie Waldo Daniels)",1st,25.00,female,0
4,"Allison, Master Hudson Trevor",1st,0.92,male,1


### Основные параметы датасета

In [112]:
print(f'Количество точек данных (измерений) равно {df.shape[0]}')
print(f'Количество полей данных (атрибутов) равно {df.shape[1]}')

Количество точек данных (измерений) равно 1313
Количество полей данных (атрибутов) равно 5


In [113]:
print('Тип данных и количество отсутствующих значений для каждого поля')
print()
df.info()

Тип данных и количество отсутствующих значений для каждого поля

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1313 entries, 0 to 1312
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Name      1313 non-null   object 
 1   PClass    1312 non-null   object 
 2   Age       756 non-null    float64
 3   Sex       1313 non-null   object 
 4   Survived  1313 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 51.4+ KB


In [114]:
print('Количество и названия константных и уникальных полей')
df.nunique()

Количество и названия константных и уникальных полей


Name        1310
PClass         3
Age           75
Sex            2
Survived       2
dtype: int64

In [115]:
[print(f'{col} -> {df[col].unique()} \n') for col in df.columns]

Name -> ['Allen, Miss Elisabeth Walton' 'Allison, Miss Helen Loraine'
 'Allison, Mr Hudson Joshua Creighton' ... 'Zenni, Mr Philip'
 'Lievens, Mr Rene' 'Zimmerman, Leo'] 

PClass -> ['1st' '2nd' nan '3rd'] 

Age -> [29.    2.   30.   25.    0.92 47.   63.   39.   58.   71.   19.     nan
 50.   24.   36.   37.   26.   28.   45.   22.   41.   48.   44.   59.
 60.   53.   33.   14.   11.   49.   46.   27.   31.   64.   55.   70.
 69.   38.   17.    4.   23.   35.   54.   21.   52.   16.   51.   42.
 40.   15.   65.   18.   56.   43.   61.   13.   34.    6.   57.   32.
 62.   67.   20.    1.   12.    0.83  8.    7.    3.    0.8   9.    5.
  0.33  0.17 10.    1.5 ] 

Sex -> ['female' 'male'] 

Survived -> [1 0] 



[None, None, None, None, None]

### Преобразование

In [188]:
# 4) Удалить из набора поля, не несущие информативной нагрузки - содержащие уникальные либо константные значения
df = df.drop(['Name'], axis=1)
df

,PClass,Age,Sex,Survived
0,1st,29.00,female,1
1,1st,2.00,female,0
2,1st,30.00,male,0
3,1st,25.00,female,0
4,1st,0.92,male,1
...,...,...,...,...
1308,3rd,27.00,male,0
1309,3rd,26.00,male,0
1310,3rd,22.00,male,0
1311,3rd,24.00,male,0


In [189]:
# 5) Заменить возраст пассажиров на год рождения (15.04.1912)
last_date = date(1912, 4, 15)

In [190]:
def age(x):
    if np.isnan(x):
        pass
    else:
        return int((last_date - timedelta(days = x*365)).strftime('%Y'))
        

In [191]:
def to_int(x):
    return int(x)

In [192]:
df['Age'] = df['Age'].apply(age)
df

,PClass,Age,Sex,Survived
0,1st,1883.0,female,1
1,1st,1910.0,female,0
2,1st,1882.0,male,0
3,1st,1887.0,female,0
4,1st,1911.0,male,1
...,...,...,...,...
1308,3rd,1885.0,male,0
1309,3rd,1886.0,male,0
1310,3rd,1890.0,male,0
1311,3rd,1888.0,male,0


In [193]:
# 6) Переименовать все названия колонок на русском языке
df.rename(columns = {'PClass': 'Класс', 'Age': 'Возраст', 'Sex':'Пол', 'Survived': 'Выживание'}, inplace = True)

In [194]:
df

,Класс,Возраст,Пол,Выживание
0,1st,1883.0,female,1
1,1st,1910.0,female,0
2,1st,1882.0,male,0
3,1st,1887.0,female,0
4,1st,1911.0,male,1
...,...,...,...,...
1308,3rd,1885.0,male,0
1309,3rd,1886.0,male,0
1310,3rd,1890.0,male,0
1311,3rd,1888.0,male,0


In [195]:
# 7) Заменить все отсутствующие данные средним значением (в случае числового поля) либо самым распространенным в категориальных полях.

In [196]:
df.groupby('Класс')['Класс'].count().sort_values(ascending = False)

Класс
3rd    711
1st    322
2nd    279
Name: Класс, dtype: int64

In [197]:
mean_age = round(df['Возраст'].mean())

In [198]:
values = {'Класс': '3rd', 'Возраст': mean_age}

In [199]:
df = df.fillna(values)

In [200]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1313 entries, 0 to 1312
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Класс      1313 non-null   object 
 1   Возраст    1313 non-null   float64
 2   Пол        1313 non-null   object 
 3   Выживание  1313 non-null   int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 41.2+ KB


In [210]:
df['Возраст'] = df['Возраст'].astype('int32')

In [211]:
df

,Класс,Возраст,Пол,Выживание
0,1st,1883,female,1
1,1st,1910,female,0
2,1st,1882,male,0
3,1st,1887,female,0
4,1st,1911,male,1
...,...,...,...,...
1308,3rd,1885,male,0
1309,3rd,1886,male,0
1310,3rd,1890,male,0
1311,3rd,1888,male,0


In [212]:
# 8) Сохранить получившийся набор данных в формате CSV в файле ФИО_Titanic.csv
df.to_csv('Котова_Екатерина_Titanic.csv', index = False)

In [ ]:
# 10) Считать файл .xlsx в новый DataFrame

In [213]:
df2 = pd.read_excel('/Users/ekaterinakotova/Desktop/2 семестр/МО/sem3/tinanic.xlsx')

In [214]:
df2

,Класс,Возраст,Пол,Выживание
0,1st,1883,female,1
1,1st,1910,female,0
2,1st,1882,male,0
3,1st,1887,female,0
4,1st,1911,male,1
...,...,...,...,...
1308,3rd,1885,male,0
1309,3rd,1886,male,0
1310,3rd,1890,male,0
1311,3rd,1888,male,0


### Контрольные вопросы

1.	Какие две главные структуры данных используются в pandas? В чем их отличие?

In [243]:
# 1 - Series - одномерный массив  
# 2 - DataFrame - двумерный массив, таблица

2.	Назовите основные функции чтения данных из файлов

In [216]:
pd.read_csv

<function pandas.io.parsers._make_parser_function.<locals>.parser_f(filepath_or_buffer: Union[str, pathlib.Path, IO[~AnyStr]], sep=',', delimiter=None, header='infer', names=None, index_col=None, usecols=None, squeeze=False, prefix=None, mangle_dupe_cols=True, dtype=None, engine=None, converters=None, true_values=None, false_values=None, skipinitialspace=False, skiprows=None, skipfooter=0, nrows=None, na_values=None, keep_default_na=True, na_filter=True, verbose=False, skip_blank_lines=True, parse_dates=False, infer_datetime_format=False, keep_date_col=False, date_parser=None, dayfirst=False, cache_dates=True, iterator=False, chunksize=None, compression='infer', thousands=None, decimal: str = '.', lineterminator=None, quotechar='"', quoting=0, doublequote=True, escapechar=None, comment=None, encoding=None, dialect=None, error_bad_lines=True, warn_bad_lines=True, delim_whitespace=False, low_memory=True, memory_map=False, float_precision=None)>

In [217]:
pd.read_excel

<function pandas.io.excel._base.read_excel(io, sheet_name=0, header=0, names=None, index_col=None, usecols=None, squeeze=False, dtype=None, engine=None, converters=None, true_values=None, false_values=None, skiprows=None, nrows=None, na_values=None, keep_default_na=True, verbose=False, parse_dates=False, date_parser=None, thousands=None, comment=None, skipfooter=0, convert_float=True, mangle_dupe_cols=True, **kwds)>

In [ ]:
pd.read_json()
pd.read_hdf()
pd.read_pickle()

3.	Напишите функцию записи набора данных А в файл с разделителями-запятыми

In [ ]:
A.to_csv('some.csv')

4.	Как называется функция, выводящая на экран первые N строчек набора данных?

In [ ]:
pandas.DataFrame.head(N)

5.	С помощью какой функции можно удалить поле набора данных? А строку?

In [ ]:
pandas.DataFrame.drop(axis = 1) - удалить поле
pandas.DataFrame.drop(axis = 0) - удалить строку

6.	Напишите участок кода, демонстрирующий возможность добавления новой строки к набору данных. 

In [ ]:
df.append(new_row, ignore_index=True) 
new_row - объект Series

7.	Напишите участок кода, демонстрирующий возможность добавления новой колонки (поля) к набору данных. 

In [ ]:
df["column_name"] = new_column
df.insert(index, "column_name", new_column)